In [1]:
import matplotlib.pyplot as plt
import numpy as np
import mpmath as mp
mp.mp.dps = 50 #higher precision
from scipy.optimize import curve_fit
import scipy.integrate as integrate
from plotDensity import *

In [2]:
def log_sum_exp(to_sum):
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value-maxval)
    res = maxval + mp.log(exp_sum)
    return res


def free_energy(E_list, log_g_list,  T):
    #Need to log sum over g(E)*exp(-E/T) without overflow issues
    to_sum = []
    for i, log_g in enumerate(log_g_list):
        to_sum.append(log_g - E_list[i]/T)
    maxval = max(to_sum)
    exp_sum = 0
    for value in to_sum:
        exp_sum += mp.exp(value-maxval)
    res = maxval + mp.log(exp_sum)
    return -T*res

def get_free_energies(rescaled_results,temperatures):
    free_energies = []
    for seed_results in rescaled_results:
        free_energy_classes = []
        for error_result in seed_results:
            f_values = []
            for T in temperatures:
                f_values.append(free_energy(error_result[0], error_result[1], T)/(-T))
            free_energy_classes.append(f_values)
        free_energies.append(free_energy_classes)
    return free_energies


def process_results(batch_results,X,Y):
    rescaled_results = []
    for seed_results in batch_results:

        rescaled_seed_results = []
        for error_result in seed_results:

            walker_results = error_result
            walker_results = get_renormalized_log_g_values_as_dict_list(walker_results)
            walker_results = average_matching_keys(walker_results)
            results_x = []
            results_y = []
            for result in walker_results:
                results_y.append(np.array(list(result.values())))
                results_x.append(np.array(list(result.keys())))

            derivatives_wrt_e = get_derivative_wrt_e(walker_results)
            minimum_deviation_energies = find_lowest_inverse_temp_deviation(derivatives_wrt_e)
            rescale_results_for_concatenation(results_x, results_y, minimum_deviation_energies)

            x_max = -1 -2*X*Y
            rescaled_x = []
            rescaled_y = []
            for i in range(len(results_x)):
                for j in range(len(results_x[i])):
                    if results_x[i][j] > x_max: #avoid double counting
                        x_max = results_x[i][j]
                        rescaled_x.append(results_x[i][j])
                        rescaled_y.append(results_y[i][j])

            offset = log_sum_exp(rescaled_y)
            rescaled_y = [res + mp.log(2)*X*Y - offset for res in rescaled_y]
            rescaled_seed_results.append([rescaled_x,rescaled_y])
        rescaled_results.append(rescaled_seed_results)
    return rescaled_results


In [3]:
from PyMatching import *

num_shots = 200000
Ls = [4,6]
ps = [0.08,0.10,0.12]
#ps = np.linspace(0.01, 0.12, 12)
np.random.seed(2)
log_errors_all_L = []
for L in Ls:
    print("Simulating L={}...".format(L))
    Hx = toric_code_x_stabilisers(L)
    logX = toric_code_x_logicals(L)
    log_errors = []
    for error_probability in ps:
        num_errors = num_decoding_failures_vectorised(Hx, logX, error_probability, num_shots)
        log_errors.append(num_errors/num_shots)
    log_errors_all_L.append(np.array(log_errors))

Simulating L=4...
Simulating L=6...


In [37]:
seed_runs = {
    "4": {
        "0.08":{
            "I": 1003,
            "X": 1000,
            "Y": 1001,
            "Z": 1002
        },
        "0.1":{
            "I": 1011,
            "X": 1008,
            "Y": 1009,
            "Z": 1010
        },
        "0.12":{
            "I": 1019,
            "X": 1016,
            "Y": 1017,
            "Z": 1018
        }
    },
    "6": {
        "0.08":{
            "I": 1007,
            "X": 1004,
            "Y": 1005,
            "Z": 1006
        },
        "0.1":{
            "I": 1015,
            "X": 1012,
            "Y": 1013,
            "Z": 1014
        },
        "0.12":{
            "I": 1023,
            "X": 1020,
            "Y": 1021,
            "Z": 1022
        }
    }
}

In [41]:
batch_results = []

probabilities = [0.08, 0.1, 0.12]

for p in probabilities:

    print('Probability',p)


    seed_range = range(1,251)

    # if p in [0.12]:
    #     seed_range = range(100,501)


    # #Taking more shots at low p
    # if p in [0.03]:
    #     seed_range = list(range(100,651)) + list(range(801,920))

    # if p in [0.01,0.02]:
    #     seed_range = range(100,1501)

    #seed_range = list(seed_range)+list(range(2000,2265))

    batch_res_4 = []

    for seed in seed_range:

        res = []
        all_good = True

        try:
            for error in ["I","X","Y","Z"]:
                seed_run = seed_runs["4"][str(p)][error]
                filename = "../results/periodic_old_handling/prob_%6f"%p+"/X_4_Y_4/seed_%d"%seed+"/error_class_"+error+"/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_%d"%seed_run+"_alpha_0.800000_beta_0.0000010000exchange_offset50.txt"
                res.append(read_data_from_file(filename))

        except:
            print('One or more error classes at 4 by 4 had an issue for seed',seed,'at probability',p)
            all_good = False

        if all_good:
            batch_res_4.append(res)

    # batch_res_6 = []
    # for seed in seed_range:

    #     res = []
    #     all_good = True

    #     try:
    #         for error in ["I","X","Y","Z"]:
                  seed_run = seed_runs["6"][str(p)][error]
    #             filename = "../results/periodic/prob_%6f"%p+"/X_6_Y_6/seed_%d"%seed+"/error_class_"+error+"/intervals_10_iterations_10000_overlap_0.250000_walkers_8_seed_run_%d"%seed+"_alpha_0.800000_beta_0.0001000000.txt"

    #             if new_run:
    #                 filename = "../results/periodic/prob_%6f"%p+"/X_6_Y_6/seed_%d"%seed+"/error_class_"+error+"/intervals_10_iterations_10000_overlap_0.250000_walkers_8_seed_run_%d"%seed1+"_alpha_0.800000_beta_0.0000010000exchange_offset1.txt"

    #                 if seed >= 400:
    #                     filename = "../results/periodic/prob_%6f"%p+"/X_6_Y_6/seed_%d"%seed+"/error_class_"+error+"/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_%d"%seed1+"_alpha_0.800000_beta_0.0000010000exchange_offset1.txt"
    #                 if seed >= 600:
    #                     filename = "../results/periodic/prob_%6f"%p+"/X_6_Y_6/seed_%d"%seed+"/error_class_"+error+"/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_0_alpha_0.800000_beta_0.0000010000exchange_offset4.txt"

    #                 filename = "../results/periodic/prob_%6f"%p+"/X_6_Y_6/seed_%d"%seed+"/error_class_"+error+"/intervals_10_iterations_1000_overlap_0.250000_walkers_8_seed_run_0_alpha_0.800000_beta_0.0000010000exchange_offset4.txt"


    #             res.append(read_data_from_file(filename))
    #     except:
    #         print('One or more error classes at 6 by 6 had an issue for seed',seed,'at probability',p)
    #         all_good = False

    #     if all_good:
    #         batch_res_6.append(res)


    # print('Adding to results for probability',p)
    # print('Number of seeds:', len(batch_res_4), len(batch_res_6))
    # rescaled_res_4 = process_results(batch_res_4,4,4)
    # rescaled_res_6 = process_results(batch_res_6,6,6)
    # batch_results.append([rescaled_res_4,rescaled_res_6])

Probability 0.08
One or more error classes at 4 by 4 had an issue for seed 251 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 252 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 253 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 254 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 255 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 256 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 257 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 258 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 259 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 260 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 261 at probability 0.08
One or more error classes at 4 by 4 had an issue for seed 262 at probability 0.08